In [1]:
codon_chart = {'AUG': 'M', 'UAA': 'STOP', 'UAG': 'STOP', 'UGA': 'STOP', 
               'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 
               'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
              'UAU': 'Y', 'UAC': 'Y', 'UGU': 'C', 'UGC': 'C', 'UGG': 'W',
              'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
              'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
              'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
              'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
              'AAU': 'I', 'AUC': 'I', 'AUA': 'I',
              'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
              'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
              'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
              'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
              'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
              'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
              'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}

dna_seq = input('Please type your DNA sequence:')
def dnaseq_aminoacidseq(dna_seq):
    rna_seq = dna_seq.replace('T', 'U') #Convert to mRNA seq
    amino_acids = []
    for i in range(0, len(rna_seq), 3): # Range from 1st to last index, in steps of 3 (bc every 3 nucleotides is 1 codon)
        codon = rna_seq[i:i+3]
        if codon in codon_chart:
            amino_acid = codon_chart[codon]
            if amino_acid == 'STOP':
                break
            amino_acids.append(amino_acid)
    amino_acids_string = ''.join([str(element) for element in amino_acids]) # Converting to string
    return amino_acids_string

print(dnaseq_aminoacidseq(dna_seq))

Please type your DNA sequence: ATGCGGAAG


MRK


In [1]:
from Bio.Blast import NCBIWWW, NCBIXML

query_sequence = "MALWMRLLPLLALLALWGPDPAAA" 

result_handle = NCBIWWW.qblast("blastp", "pdb", query_sequence)

blast_records = NCBIXML.parse(result_handle)

for blast_record in blast_records:
    print(f"Query: {blast_record.query}")
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print(f"Hit: {alignment.title}")
            print(f"Score: {hsp.score}, E-value: {hsp.expect}")


Query: unnamed protein product
Hit: pdb|6B3Q|a Chain a, Insulin [Homo sapiens] >pdb|6B3Q|b Chain b, Insulin [Homo sapiens] >pdb|6B70|a Chain a, Insulin [Homo sapiens] >pdb|6B70|c Chain c, Insulin [Homo sapiens] >pdb|6BFC|a Chain a, Insulin [Homo sapiens] >pdb|6BFC|b Chain b, Insulin [Homo sapiens] >pdb|6JK8|C Chain C, Insulin [Homo sapiens] >pdb|6JK8|D Chain D, Insulin [Homo sapiens] >pdb|6TC2|A Chain A, Insulin [Homo sapiens] >pdb|6TC2|B Chain B, Insulin [Homo sapiens] >pdb|6TC2|C Chain C, Insulin [Homo sapiens] >pdb|6TC2|D Chain D, Insulin [Homo sapiens] >pdb|6TC2|E Chain E, Insulin [Homo sapiens] >pdb|6TC2|F Chain F, Insulin [Homo sapiens] >pdb|6TC2|G Chain G, Insulin [Homo sapiens] >pdb|6TC2|H Chain H, Insulin [Homo sapiens] >pdb|6TC2|I Chain I, Insulin [Homo sapiens] >pdb|6TC2|J Chain J, Insulin [Homo sapiens] >pdb|6TC2|K Chain K, Insulin [Homo sapiens] >pdb|6TC2|L Chain L, Insulin [Homo sapiens] >pdb|7STH|C Chain C, Insulin [Homo sapiens] >pdb|7STH|D Chain D, Insulin [Homo sapien